In [1]:
import pandas as pd
import numpy as np 


from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data/ABT_partition4.csv')

In [3]:
df

,class,active_region,start_time,end_time,TOTUSJH_Max,TOTUSJH_Min,TOTUSJH_Median,TOTUSJH_Mean,TOTUSJH_Std_dev,TOTUSJH_Variance,...,R_VALUE_Mean,R_VALUE_Std_dev,R_VALUE_Variance,R_VALUE_Skewness,R_VALUE_Kurtosis,R_VALUE_avg_abs_derivative_change,R_VALUE_last_value,R_VALUE_average_absolute_change,R_VALUE_quadratic_weighted_moving_average,R_VALUE_linear_weighted_moving_average
0,1,4874,2014-12-04T03:24:00,2014-12-04T15:12:00,2785.852833,2491.531685,2641.136226,2620.825580,90.493670,8189.104400,...,4.654618,0.085263,0.007270,-0.324078,-1.191564,0.003174,4.774954,0.023202,4.712566,4.695854
1,1,5011,2015-01-03T17:36:00,2015-01-04T05:24:00,3219.186282,2806.196014,2931.875374,2958.810152,105.426848,11114.820218,...,4.624520,0.112293,0.012610,-0.216595,-1.097424,0.003077,4.454785,0.024048,4.531087,4.563562
2,1,4781,2014-11-05T17:00:00,2014-11-06T04:48:00,4240.705267,3548.885212,3889.603145,3864.781382,224.197421,50264.483419,...,5.325489,0.016920,0.000286,0.343941,0.095240,0.001215,5.316678,0.010504,5.318351,5.319844
3,1,4781,2014-11-05T22:00:00,2014-11-06T09:48:00,4384.021730,3866.071556,4111.197138,4119.445572,158.696759,25184.661170,...,5.294639,0.040382,0.001631,-1.594406,1.613134,0.001089,5.177362,0.009298,5.264564,5.278021
4,1,4781,2014-11-08T15:00:00,2014-11-09T02:48:00,3598.551137,3122.532031,3397.790137,3365.688336,154.227212,23786.032956,...,4.790613,0.031215,0.000974,0.312492,-0.321891,0.001609,4.752749,0.012820,4.770827,4.779305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51256,1,4272,2014-06-24T18:36:00,2014-06-25T06:24:00,992.661572,852.157091,949.046185,935.832556,37.344589,1394.618351,...,4.347464,0.079230,0.006277,-0.365231,-0.543393,0.004014,4.158737,0.030818,4.286448,4.307739
51257,0,5006,2014-12-28T10:00:00,2014-12-28T21:48:00,12.656314,6.097119,9.315705,9.356092,1.636295,2.677460,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
51258,0,4469,2014-08-26T21:48:00,2014-08-27T09:36:00,322.016807,226.421381,298.731413,290.855592,25.632848,657.042912,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
51259,0,5020,2014-12-31T05:48:00,2014-12-31T17:36:00,141.255501,1.032144,113.458711,99.417747,33.979757,1154.623911,...,1.610879,1.241859,1.542213,-0.409658,-1.620817,0.081091,0.000000,0.538215,1.155247,1.390143


In [4]:
model_loaded = XGBClassifier()
model_loaded.load_model("../models/XGBoost/XGBoostModel_v2.ubj")

In [5]:
# top_25_features = ['R_VALUE_Min',
#  'TOTUSJH_avg_abs_derivative_change',
#  'TOTPOT_Variance',
#  'TOTUSJH_last_value',
#  'ABSNJZH_Min',
#  'TOTFZ_Min',
#  'SAVNCPP_Variance',
#  'TOTFZ_average_absolute_change',
#  'TOTPOT_Std_dev',
#  'TOTPOT_last_value',
#  'TOTUSJH_Variance',
#  'USFLUX_last_value',
#  'TOTUSJH_Max',
#  'TOTFZ_last_value',
#  'TOTUSJZ_last_value',
#  'MEANGBZ_Max',
#  'ABSNJZH_average_absolute_change',
#  'SAVNCPP_average_absolute_change',
#  'TOTUSJH_Std_dev',
#  'MEANPOT_Mean',
#  'R_VALUE_Variance',
#  'ABSNJZH_Variance']

In [6]:
Y = df.copy()['class']
X = df.copy().drop(['class','active_region'],axis=1)#undo
GROUP = df.copy()['active_region']

In [10]:
import pickle
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from xgboost import XGBClassifier

# load model + scaler + features
with open('../models/XGBoost/scaler_v2.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

loaded_model = XGBClassifier()
loaded_model.load_model('../models/XGBoost/XGBoostModel_v2.ubj')

with open('../models/XGBoost/feature_names_v2.pkl', 'rb') as f:
    feature_names = pickle.load(f)

print("Model, scaler, and feature names loaded!")

# prep data, we have to use the same scaling constants from training
#loading a new partition
new_data = X.copy()

# Ensure features are in correct order
new_data = new_data[feature_names]

#scale data using transform dont call fit 
new_data_scaled = pd.DataFrame(
    loaded_scaler.transform(new_data),
    columns=new_data.columns,
    index=new_data.index
)

# y_pred
predictions = loaded_model.predict(new_data_scaled)
predictions_proba = loaded_model.predict_proba(new_data_scaled)[:, 1]

# perfo metrics 
print("\n" + "="*50)
print("PERFORMANCE METRICS ON UNSEEN PARTITION")
print("="*50)

print("\nClassification Report:")
print(classification_report(Y, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(Y, predictions))

print(f"\nROC AUC Score: {roc_auc_score(Y, predictions_proba):.4f}")

# output sample preds
print("\n" + "="*50)
print("SAMPLE PREDICTIONS")
print("="*50)
results_df = pd.DataFrame({
    'True_Class': Y.values,
    'Predicted_Class': predictions,
    'Flare_Probability': predictions_proba
})
print(results_df)

Model, scaler, and feature names loaded!

PERFORMANCE METRICS ON UNSEEN PARTITION

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.94      0.94     43294
           1       0.67      0.63      0.65      7967

    accuracy                           0.89     51261
   macro avg       0.80      0.79      0.79     51261
weighted avg       0.89      0.89      0.89     51261


Confusion Matrix:
[[40767  2527]
 [ 2937  5030]]

ROC AUC Score: 0.9137

SAMPLE PREDICTIONS
       True_Class  Predicted_Class  Flare_Probability
0               1                1           0.991273
1               1                1           0.999668
2               1                1           0.996242
3               1                1           0.999686
4               1                1           0.978859
...           ...              ...                ...
51256           1                0           0.032775
51257           0                0        

In [8]:
results_df

,True_Class,Predicted_Class,Flare_Probability
0,1,1,0.991273
1,1,1,0.999668
2,1,1,0.996242
3,1,1,0.999686
4,1,1,0.978859
...,...,...,...
51256,1,0,0.032775
51257,0,0,0.000044
51258,0,0,0.006242
51259,0,0,0.000284


In [9]:
feature_names

['R_VALUE_Min',
 'TOTUSJH_last_value',
 'TOTUSJH_avg_abs_derivative_change',
 'TOTFZ_Min',
 'SAVNCPP_Variance',
 'TOTUSJH_Max',
 'TOTFZ_average_absolute_change',
 'TOTPOT_Variance',
 'TOTFZ_last_value',
 'ABSNJZH_Min',
 'SAVNCPP_average_absolute_change',
 'ABSNJZH_average_absolute_change',
 'TOTUSJH_Variance',
 'TOTPOT_Std_dev',
 'R_VALUE_Variance',
 'MEANPOT_Mean',
 'USFLUX_last_value',
 'TOTFZ_Variance',
 'TOTBSQ_Variance',
 'ABSNJZH_Variance',
 'MEANGBZ_Max',
 'TOTUSJZ_last_value']